# DLO-JZ Optimisation de l'apprentissage - Jour 2 matin

Optimisation système d'une boucle d'apprentissage *Resnet-50*.

![car](./images/optimisation.png)

## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Resnet-50* sur *Imagenet* pour Jean Zay en implémentant :
* **TP 1** : la distribution (*Data Parallelism*)
* **TP 2** : Imagenet Race - Test Tour

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant le code `dlojz.py`.

Les directives de modification seront marquées par l'étiquette **TODO :**, dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, juin 2023*


------------------------

### Environnement de calcul

Un module PyTorch doit avoir été chargé pour le bon fonctionnement de ce Notebook. **Nécessairement**, le module `pytorch-gpu/py3/1.11.0` :

In [ ]:
!module list

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter
MODULE = 'pytorch-gpu/py3/1.11.0'
image_size = 176
account = 'for@v100'
name = 'pseudo'   ## Pseudonyme à choisir

Creation d'un repertoire `checkpoints` si cela n'a pas déjà été fait.

In [ ]:
!mkdir checkpoints

------------------------------------

### Gestion de la queue SLURM

Cette partie permet d'afficher et de gérer la queue SLURM.

Pour afficher toute la queue *utilisateur* :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction utilisée plusieurs fois dans ce *notebook* permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Cependant elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact sur le *scheduler* SLURM. Les *jobs* ne seront pas arrêtés.

Si vous voulez arrêter des *jobs* dans la queue:
* Annuler tous vos *jobs* dans la queue (décommenter la ligne suivante) : `!scancel -u $USER`
* Annuler un *job* dans votre queue (décommenter la ligne suivante et ajouter le numéro du *job* à la fin de la ligne)


In [ ]:
#!scancel -u $USER

------------------------------------

### Debug

Cette partie *debug* permet d'afficher les fichiers de sortie et les fichiers d'erreur du *job*.

Il est nécessaire dans la cellule suivante (en décommentant) d'indiquer le *jobid* correspondant sous le format suivant.

***Remarque*** : dans ce notebook, lorsque vous soumettrez un *job*, vous recevrez en retour le numéro du job dans le format suivant : `jobid = ['123456']`. La cellule ci-dessous peut ainsi être facilement actualisée."

In [ ]:
jobid = ['1493206']

Fichier de sortie :

In [ ]:
%cat {search_log(contains=jobid[0])[0]}

Fichier d'erreur :

In [ ]:
%cat {search_log(contains=jobid[0], with_err=True)['stderr'][0]}

--------------

### Différence entre deux scripts

Pour le *debug* ou pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page html contenant un différentiel de fichiers texte.

In [ ]:
s1 = "dlojz.py"
s2 = "./solutions/dlojz2_1.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

----------------------

## Garage - Mise à niveau

On fixe la taille d'image pour ce TP.

In [ ]:
image_size = 176

On choisit le *batch size* optimal d'après les expériences du Jour 1.

In [ ]:
## Choisir un batch size optimal
bs_optim = 512   ##TODO

**TODO :** Comparer votre script `dlojz.py` avec ce qu'il devrait être actuellement. Si il y a des divergences, veuillez les corriger (par exemple en copiant-collant la solution).

In [ ]:
s1 = "dlojz.py"
s2 = "./solutions/dlojz2_0.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante :

[compare.html](compare.html)

In [ ]:
# copier/coller la solution si nécessaire
#!cp solutions/dlojz2_0.py dlojz.py

----------------------------------------------
# TP2_1 : Distribution - Parallélisme de données

**TODO** : dans le script `dlojz.py` :
* Importer les fonctionnalités liées à la distibution et au *Data Parallelism*.
```python
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
```

* Configurer la distribution.

```python
    # configure distribution method: define rank and initialise communication backend (NCCL)
    dist.init_process_group(backend='nccl', init_method='env://',
                            world_size=idr_torch.size, rank=idr_torch.rank)
```

* Associer le GPU alloué au *process*.
```python
    # define model
    torch.cuda.set_device(idr_torch.local_rank)
    gpu = torch.device("cuda")
```

* Mettre en place la distribution du modèle.
```python
    model = DistributedDataParallel(model, device_ids=[idr_torch.local_rank])
```


* Dans la partie *DATALOADER*, 
  * Implémenter les *samplers* pour les `train_loader` et `val_loader`.
```python
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset,
                                                                    num_replicas=idr_torch.size,
                                                                    rank=idr_torch.rank,
                                                                    shuffle=True)
```
```python
    val_sampler = torch.utils.data.distributed.DistributedSampler(val_dataset,
                                                                  num_replicas=idr_torch.size,
                                                                  rank=idr_torch.rank,
                                                                  shuffle=False)
```

* Dans `train_loader` et `val_loader`, ajouter la ligne `sampler=train_sampler,` (ou `sampler=val_sampler,`) pour associer le bon *sampler* et basculer le `shuffle=` du *loader* à ` False` pour ne pas avoir de conflit avec le *shuffle* du *sampler*.
  
  
* Au tout début de la boucle d'apprentissage indiquer au *sampler* l'*epoch*, afin d'obtenir un *shuffle* différent à chaque *epoch*.
```python
    #### TRAINING ############
    for epoch in range(args.epochs):
        train_sampler.set_epoch(epoch)
```

* Les métriques pour l'apprentissage doivent être échangées et moyennées.
```python
    # Metric mesurement
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == labels).sum() / labels.size(0)
    dist.all_reduce(accuracy, op=dist.ReduceOp.SUM)
    accuracy /= idr_torch.size
    if idr_torch.rank == 0: accuracies.append(accuracy.item())
```

* Les métriques pour la validation doivent être échangées et moyennées **après la boucle** de validation.
```python
    for iv, (val_images, val_labels) in enumerate(val_loader):
        ...
        ...
    dist.all_reduce(val_loss, op=dist.ReduceOp.SUM)
    dist.all_reduce(val_accuracy, op=dist.ReduceOp.SUM)
```

**A noter** : la moyenne des métriques distribuées sur les différents GPU se calcule pour la validation différemment du *training*. Ici la métrique est pondérée par rapport à la taille globale du dataset de validation. Il n'est donc pas nécessaire de diviser par `idr_torch.size`.

* Ajouter une barrière après la boucle d'apprentissage afin d'éviter que certains *process* sortent de la distribution à la toute fin, alors que d'autres n'ont pas fini leur boucle de validation.
```python
## Be sure all process finish at the same time to avoid incoherent logs at the end of process
dist.barrier()
```

In [ ]:
command = f'dlojz.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493733']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

------------------------

# TP2_2 : Imagenet Racing - Test Tour 

![race](./images/F1.png)


Le but de ce TP est de paramétrer l'entraînement pour participer à la course Imagenet Racing.

Les *job* de chaque participant durant environ 30 minutes, s'exécuteront pendant la nuit. Les résultats seront commentés le lendemain.

In [ ]:
from dlojz_tools import plot_accuracy, imagenet_starter, plot_time

**TODO :** Chercher les bons paramètres, notamment la *taille de batch par GPU*  `batch_size` et la *taille d'image* `image_size` permettant d'avoir un bon équilibre (d'après votre intuition) entre une taille d'image suffisante et un nombre d'*epochs* suffisant.

Le nombre d'*epochs* auquel vous avez le droit dépend du *Throughput* mesuré pendant le test. Il faudra regarder la dernière ligne du test `Eligible to run X epochs` pour connaître cette mesure.


#### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
## TODO : Définir une taille d'image
image_size = 176

In [ ]:
#jobids = ['1493910', '1493914', '1493916', '1493918', '1493920', '1493932', '1493937']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

In [ ]:
## TODO : Choisir un batch size optimal
batch_size = 512

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Nous vous conseillons de garder plusieurs lignes en mémoire afin de pouvoir comparer facilement vos différentes expériences.

In [ ]:
#jobid = ['1494033']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Apprentissage complet sur 32 GPU (à lancer en toute fin de journée)

**TODO :** Une fois que vous avez choisi la configuration que vous souhaitez engager pour la course, la fonction suivante permet de générer la bonne commande à soumettre à *SLURM* avec le bon nombre d'*epochs*, les bonnes configurations de *taille de batch par GPU*  et de *taille d'image*, à condition d'avoir fourni le bon `jobid`.

In [ ]:
?imagenet_starter

In [ ]:
#jobid = ['1292862']

In [ ]:
command = imagenet_starter(jobid, weight_decay=5e-4)
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1494173']

### Visualisation des résultats

In [ ]:
jobids = ['1494173', jobid[0]]

#### Résultat de référence : image_size = 176, batch_size = 512

In [ ]:
plot_accuracy(jobids[:1])

In [ ]:
plot_time(jobids[:1])

In [ ]:
display_slurm_queue(name+'_race')

#### Votre résultat

In [ ]:
plot_accuracy(jobids)

In [ ]:
plot_time(jobid)

### Publication des Résultats sur WandB

Décommenter la ligne `#!wandb sync --sync-all` pour publier les résultats sur le dépôt WandB

In [ ]:
import os
os.environ['WANDB_API_KEY']='2ecf1cc3a3fe45c17b480e66dd0f390c85763d42'
#!wandb sync --sync-all

https://wandb.ai/dlojz/Imagenet%20Race%20Cup?workspace=user-bcabot

--------------